<pre>
   _____                            _                __      ___     _                  _                        _ 
  / ____|                          | |               \ \    / (_)   (_)                | |                      | |
 | |     ___  _ __ ___  _ __  _   _| |_ ___ _ __      \ \  / / _ ___ _  ___  _ __      | |__   __ _ ___  ___  __| |
 | |    / _ \| '_ ` _ \| '_ \| | | | __/ _ \ '__|      \ \/ / | / __| |/ _ \| '_ \     | '_ \ / _` / __|/ _ \/ _` |
 | |___| (_) | | | | | | |_) | |_| | ||  __/ |          \  /  | \__ \ | (_) | | | |    | |_) | (_| \__ \  __/ (_| |
  \_____\___/|_| |_| |_| .__/ \__,_|\__\___|_|           \/   |_|___/_|\___/|_| |_|    |_.__/ \__,_|___/\___|\__,_|
  _______              | | __  __                                                    _                             
 |__   __|             |_||  \/  |                                                  | |                            
    | |_ __ ___  ___      | \  / | ___  __ _ ___ _   _ _ __ ___ _ __ ___   ___ _ __ | |_                           
    | | '__/ _ \/ _ \     | |\/| |/ _ \/ _` / __| | | | '__/ _ \ '_ ` _ \ / _ \ '_ \| __|                          
    | | | |  __/  __/     | |  | |  __/ (_| \__ \ |_| | | |  __/ | | | | |  __/ | | | |_                           
    |_|_|  \___|\___|     |_|  |_|\___|\__,_|___/\__,_|_|  \___|_| |_| |_|\___|_| |_|\__|                          
                                                                                                                   
                                                                                                                   

# Global Settings

In [ ]:



## CONSTANTS

# Assets folder
ASSETS = "./assets"

# Original video's
CALIBRATION_VIDEO     = f"{ASSETS}/original/computervisie_2024/calibration.MP4"
EASTBOUND_VIDEO       = f"{ASSETS}/original/computervisie_2024/eastbound_20240319.MP4"
WESTBOUND_VIDEO       = f"{ASSETS}/original/computervisie_2024/westbound_20240319.MP4"


# Extracted frames
EXTRACTED_CALIBRATION_PATH  = f"{ASSETS}/extracted_30/calibration/"
EXTRACTED_WESTBOUND_PATH    = f"{ASSETS}/extracted_05/westbound_20240319/"
EXTRACTED_EASTBOUND_PATH    = f"{ASSETS}/extracted_05/eastbound_20240319/"
EXTRACTED_CALIBRATION_FILES = f"{EXTRACTED_CALIBRATION_PATH}*.png"
EXTRACTED_WESTBOUND_FILES   = f"{EXTRACTED_WESTBOUND_PATH}*.png"
EXTRACTED_EASTBOUND_FILES   = f"{EXTRACTED_EASTBOUND_PATH}*.png"


# Undistorted frames
UNDISTORTED_WESTBOUND_PATH  = f"{ASSETS}/undistorted_05/westbound/"
UNDISTORTED_EASTBOUND_PATH  = f"{ASSETS}/undistorted_05/eastbound/"
UNDISTORTED_WESTBOUND_FILES = f"{ASSETS}/undistorted_05/westbound/*.png"
UNDISTORTED_EASTBOUND_FILES = f"{ASSETS}/undistorted_05/eastbound/*.png"


# Annotated frames
ANNOTATED_WESTBOUND = f"{ASSETS}/annotated_05/westbound/"
ANNOTATED_EASTBOUND = f"{ASSETS}/annotated_05/eastbound/"


# PercepTree Model
PERCEPTREE_MODEL = f"{ASSETS}/models/ResNext-101_fold_01.pth"


# Colmap Workspaces
COLMAP_WORKSPACE_WESTBOUND = f"{ASSETS}/colmap/workspaces/westbound/"
COLMAP_WORKSPACE_EASTBOUND = f"{ASSETS}/colmap/workspaces/eastbound/"

# temp local
#COLMAP_WORKSPACE_EASTBOUND = f"C:/2023-2024/M_ComputerVisie/colmap_sparse_and_dense_and_text/"



# Colmap TXT reconstructions
COLMAP_TXT_RECON_EASTBOUND = f"{ASSETS}/colmap/reconstruction/eastbound/"
COLMAP_TXT_RECON_WESTBOUND = f"{ASSETS}/colmap/reconstruction/westbound/"


# Preliminary

## Installs

In [ ]:
!pip install numpy
!pip install gdown
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install torch



# When running code in JupyterHub / Google Colab, Opencv might not be
# installed. This piece of code installs it when it is not yet available.
try:
    import cv2
except ImportError:
    print("OpenCV is not installed, installing now")
    !pip install opencv-python
    import cv2

from __future__ import  absolute_import


is_linux = False
if is_linux:
    # detectron 2 install & imports
    !pip install "git+https://github.com/facebookresearch/detectron2.git"
    # NOTE: Doesn't seem to work on windows?

    # Setup detectron2 logger
    from detectron2.utils.logger import setup_logger
    setup_logger()

## Imports

In [ ]:
# general imports
import os
import gc
import cv2
import json
import torch
import random
import requests
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from pathlib import Path
from zipfile import ZipFile

# matplotlib imports
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors as mcolors
import matplotlib.patches as patches
import matplotlib.animation as animation
from matplotlib.colors import LinearSegmentedColormap


# other visualisation tool imports
# import open3d as o3d
from IPython.display import HTML
from mpl_toolkits.mplot3d import Axes3D


# sklearn imports
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

if is_linux:
    # detectron2 utilities imports
    from detectron2 import model_zoo
    from detectron2.config import get_cfg
    from detectron2.data import MetadataCatalog
    from detectron2.engine import DefaultPredictor
    from detectron2.utils.visualizer import Visualizer
    from detectron2.utils.video_visualizer import VideoVisualizer

## Download Source Video's

In [ ]:
# Eastbound Video
file_path = Path(EASTBOUND_VIDEO)

# If video does not exist, we need to download the video's
if not file_path.exists():
    print("Downloading video's")

    url = "https://telin.ugent.be/nextcloud/index.php/s/rjgf4cw7m2iTGbx/download"

    response = requests.get(url, stream=True)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Open a file in binary write mode
        with open(f"{ASSETS}/videos.zip", "wb") as file:
            # Iterate over the response content by chunks
            for chunk in response.iter_content(chunk_size=1024):
                # Write the chunk to the file
                file.write(chunk)
    
        print("Zip file downloaded successfully.")
    
        with ZipFile(f"{ASSETS}/videos.zip", "r") as zip_ref:
            zip_ref.extractall(f"{ASSETS}/original")
    else:
        print("Failed to download the zip file.")
else:
    print("Video's already exist, skipping download")

## Download PercepTree Model

In [ ]:
file_path = Path(PERCEPTREE_MODEL)

if not file_path.exists():
    print('Downloading model')
    !gdown --fuzzy 'https://drive.google.com/file/d/108tORWyD2BFFfO5kYim9jP0wIVNcw0OJ/view' -O assets/models/
else:
    print('Model already downloaded, skipping download')

## Download & Install Colmap

In [ ]:
!sudo apt-get update

!sudo apt-get install \
    git \
    cmake \
    ninja-build \
    build-essential \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libeigen3-dev \
    libflann-dev \
    libfreeimage-dev \
    libmetis-dev \
    libgoogle-glog-dev \
    libgtest-dev \
    libsqlite3-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev \
    libceres-dev -y

In [ ]:
!nvidia-smi --query-gpu=compute_cap --format=csv

In [ ]:
!sudo apt-get install gcc-10 g++-10 -y
!export CC=/usr/bin/gcc-10
!export CXX=/usr/bin/g++-10
!export CUDAHOSTCXX=/usr/bin/g++-10

In [ ]:
!git clone https://github.com/colmap/colmap.git ./assets/colmap/colmap

In [ ]:
!conda remove glog -y

In [ ]:
!mkdir -p ./assets/colmap/colmap_build
!cd assets/colmap/colmap_build && ls
!cd assets/colmap/colmap_build && cmake ../colmap -GNinja -DCMAKE_CUDA_ARCHITECTURES=61 -DGUI_ENABLED=OFF

In [ ]:
!cd assets/colmap/colmap_build && ninja
!cd assets/colmap/colmap_build && sudo ninja install

# Image Processing
## Frame Extraction

In this section, we extract individual frames from the video. This process involves loading the video file, iterating through each $k$ frames, and saving these frames as separate image files. Each extracted frame is named to include its frame number, ensuring a clear and organized sequence. Extracting frames allows for detailed analysis and processing of each moment captured in the video.

In [ ]:
# frame extraction from video (keeping the original frame number)
# this could help us in the case where we cannot detect trees in certain frames, then we can use the original frame number to extract more frames from the original video in this time area

def extract_frames(video_path, capture_every_frame=5, output_folder=F"{ASSETS}/extracted"):
    # Adjust the output folder to include how many frames were skipped
    output_folder += f"_{capture_every_frame:02d}"

    # Skip if video has already been extracted
    if os.path.isdir(output_folder):
        print("skipping, video already extracted")
        return
    
    # Extract video name from path
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    
    # Create the directory if it doesn't exist
    os.makedirs(f"{output_folder}/{video_name}", exist_ok=True)

    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get total number of frames
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in {video_name}: {total_frames}")

    # Initialize the frame counter
    current_frame = 0

    # Process frames
    while current_frame < total_frames:
        video.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        success, image = video.read()

        # Check if the frame was successfully read
        # sometimes this fails, corrupt frames in vide? idk
        # using frames:05d (file_name_frame_00001.png) for easy sorting later on
        if success:
            # Save the frame with the actual frame number in the file name
            cv2.imwrite(f"{output_folder}/{video_name}/{video_name}_{current_frame:05d}.png", image)
            print(f"{output_folder}/{video_name}/{video_name}_{current_frame:05d}.png")
            # Skip to the next frame based on the specified interval
            current_frame += capture_every_frame
        else:
            # Output an error message if the frame failed to extract
            print(f"Frame {current_frame} failed to extract")
            # Try the next frame instead of skipping the interval because we couldn't read the current frame
            current_frame += 1


    # Release the video capture object
    video.release()


In [ ]:
extract_frames(CALIBRATION_VIDEO, capture_every_frame=30)

In [ ]:
extract_frames(EASTBOUND_VIDEO)

In [ ]:
extract_frames(WESTBOUND_VIDEO)

## Calibration

In this section, we calculate the camera calibration matrices using Zhang's method, which is a widely used technique for camera calibration in computer vision. This method leverages multiple images of a known calibration pattern, in this case a chessboard, to estimate the camera's intrinsic and extrinsic parameters. By utilizing Zhang's method, this calibration process enables accurate determination of the camera's parameters, which are essential for correcting lens distortion and improving the accuracy of subsequent image processing tasks.

In [ ]:
def calibrate_camera(calibration_images):
    objp = np.zeros((6*8, 3), np.float32)
    objp[:, :2] = np.mgrid[0:8, 0:6].T.reshape(-1, 2)

    objpoints = []
    imgpoints = []


    for img_path in calibration_images:
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        ret, corners = cv2.findChessboardCorners(gray, (8, 6), None)

        if ret:
            objpoints.append(objp)
            imgpoints.append(corners)


    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return ret, mtx, dist, rvecs, tvecs

In [ ]:
calibration_images = glob(EXTRACTED_CALIBRATION_FILES)

mtx_file = f"{ASSETS}/calibration_data/intrinsic_parameters.npy"
dist_file = f"{ASSETS}/calibration_data/distortion_coefficients.npy"
rvecs_file = f"{ASSETS}/calibration_data/rotation_vectors.npy"
tvecs_file = f"{ASSETS}/calibration_data/translation_vectors.npy"

if os.path.isdir(f"{ASSETS}/calibration_data"):
    print("skipping, callibration matrix already extracted")

    mtx = np.load(mtx_file)
    dist = np.load(dist_file)
    rvecs = np.load(rvecs_file)
    tvecs = np.load(tvecs_file)

else:
    print("Calculating callibration data...")
    ret, mtx, dist, rvecs, tvecs = calibrate_camera(calibration_images)

    # save data for later use
    os.makedirs(f"{ASSETS}/calibration_data", exist_ok=True)
    np.save(mtx_file, mtx)
    np.save(dist_file, dist)
    np.save(rvecs_file, rvecs)
    np.save(tvecs_file, tvecs)

## Undistortion
After calibrating the camera and obtaining the necessary calibration matrices, the next step is to undistort the images. Lens distortion, which manifests as warping or curving of straight lines in images, is a common issue in photography, especially with wide-angle lenses. Using the calibration data derived from Zhang's method, we can correct this radial distortion and produce geometrically accurate images.

In [ ]:
def undistort_images(images, output_loc=f"{ASSETS}/undistorted"):
    os.makedirs(f"{output_loc}", exist_ok=True)

    for img_path in images:
        img = cv2.imread(img_path)
        frame_name = os.path.splitext(os.path.basename(img_path))[0]
        
        undistorted_image = cv2.undistort(img, mtx, dist, None, mtx)
        cv2.imwrite(f"./{output_loc}/{frame_name}.png", undistorted_image)

In [ ]:
westbound_images = glob(EXTRACTED_WESTBOUND_FILES)
eastbound_images = glob(EXTRACTED_EASTBOUND_FILES)

westbound_images.sort()
eastbound_images.sort()

undistort_images(westbound_images, output_loc=UNDISTORTED_WESTBOUND_PATH)
undistort_images(eastbound_images, output_loc=UNDISTORTED_EASTBOUND_PATH)

# Tree Detection - TODO Clean Up

## PercepTree



In [ ]:


def detect_trees(base_path, output_dir, model_name=PERCEPTREE_MODEL, display_image=False ):
    
    # Ensure that the output directory exists, create it if necessary
    os.makedirs(output_dir, exist_ok=True)
    
    image_paths = glob(image_dir_pattern)
    image_paths.sort()
    print("Images to process:", len(image_paths))
    
    # def process_list_of_images():
    torch.cuda.is_available()
    logger = setup_logger(name=__name__)
    
    # All configurables are listed in /repos/detectron2/detectron2/config/defaults.py        
    cfg = get_cfg()
    cfg.INPUT.MASK_FORMAT = "bitmask"
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_X_101_32x8d_FPN_3x.yaml"))
    # cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml"))
    # cfg.merge_from_file(model_zoo.get_config_file("COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = ()
    cfg.DATASETS.TEST = ()
    cfg.DATALOADER.NUM_WORKERS = 8
    cfg.SOLVER.IMS_PER_BATCH = 8
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256   # faster (default: 512)
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (tree)
    cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 1  
    cfg.MODEL.ROI_KEYPOINT_HEAD.NUM_KEYPOINTS = 5
    cfg.MODEL.MASK_ON = True
    
    cfg.OUTPUT_DIR = './output'
    # cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, model_name)
    cfg.MODEL.WEIGHTS = PERCEPTREE_MODEL
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
    # cfg.INPUT.MIN_SIZE_TEST = 0  # no resize at test time
    
    # set detector
    predictor_synth = DefaultPredictor(cfg)    
    
    # set metadata
    tree_metadata = MetadataCatalog.get("my_tree_dataset").set(thing_classes=["Tree"], keypoint_names=["kpCP", "kpL", "kpR", "AX1", "AX2"])
    
    for image_path in image_paths:    
        file_name = image_path.split("/")[-1]
        output_path = output_dir + file_name
        # inference
        im = cv2.imread(image_path)
        outputs_pred = predictor_synth(im)
        v_synth = Visualizer(im[:, :, ::-1],
                        metadata=tree_metadata, 
                        scale=1,
        )
        predictions = outputs_pred["instances"].to("cpu")
        out_synth = v_synth.draw_instance_predictions(predictions)
    
        # Assuming out_synth.get_image() returns the image
        image = out_synth.get_image()[:, :, ::-1]  # Assuming the image is in BGR format, converting it to RGB
        
        # Save the image
        cv2.imwrite(output_path, image)
        print("Succesfully wrote image to:", output_path)
        
        # Convert the tensors to lists
        pred_boxes_list = predictions.get("pred_boxes").tensor.tolist()
        scores_list = predictions.get("scores").tolist()
        pred_keypoints_list = predictions.get("pred_keypoints").tolist()
        
        # .tolist() on a tensor is extremely slow, so saved it as npy instead (you can test this in a seperate cell and see how slow it is)
        # pred_masks_list = predictions.get("pred_masks_list").tolist()
        # pred_keypoints_heatmaps_list = predictions.get("pred_keypoint_heatmaps").tolist()
        pred_mask_numpy = predictions.get("pred_masks").numpy()
        pred_keypoint_heatmaps_numpy = predictions.get("pred_keypoint_heatmaps").numpy()
    
        # File base name
        file_base_name = file_name.split(".png")[0]
        # File names are kept in json as reference to numpy array file
        pred_masks_file_name = file_base_name + '_pred_mask.npy'
        pred_keypoint_heatmaps_file_name = file_base_name + '_pred_keypoints_heatmaps.npy'
        # Create full path that is used for saving the .npy files
        pred_mask_numpy_path = output_dir + pred_masks_file_name
        pred_keypoint_heatmaps_numpy_path = output_dir + pred_keypoint_heatmaps_file_name
        # Save the .npy arrays
        print("Saving:", pred_mask_numpy_path)
        np.save(pred_mask_numpy_path, pred_mask_numpy)
        print("Saving:", pred_keypoint_heatmaps_numpy_path)
        np.save(pred_keypoint_heatmaps_numpy_path, pred_keypoint_heatmaps_numpy)
    
        # Prepare a dictionary for JSON serialization
        json_data = {
            "pred_boxes": pred_boxes_list,
            "scores": scores_list,
            "pred_keypoints": pred_keypoints_list,
            "pred_masks": pred_masks_file_name,
            "pred_keypoint_heatmaps": pred_keypoint_heatmaps_file_name,
        }
        
        # Save to a JSON file    
        file_base_name = file_name.split(".png")[0]
        json_path = output_dir + file_base_name + '.json'
        with open(json_path, 'w') as json_file:
            json.dump(json_data, json_file, indent=4)
        
        print(f"Data has been saved to {json_path}")    
    
        if display_image:        
            plt.figure(figsize=(20, 10))
            plt.imshow(out_synth.get_image())
            # plt.axis('off')  # Turn off axis
            plt.show()
    
        collected_garbage = gc.collect()
        print("Removed garbage:", collected_garbage)


In [ ]:
detect_trees(UNDISTORTED_EASTBOUND_FILES, ANNOTATED_EASTBOUND)
detect_trees(UNDISTORTED_WESTBOUND_FILES, ANNOTATED_WESTBOUND)

# Tree Triangulation

## Creating Models

### Sparse model

In [ ]:
def colmap_reconstruction(workspace, image_path):
    !mkdir -p {workspace}
    !colmap automatic_reconstructor --workspace_path {workspace} --image_path {image_path} --sparse 1 --dense 0 --single_camera 1

def convert_to_txt(input, output):
    #!mkdir -p {output}
    !colmap model_converter --input_path {input} --output_path {output} --output_type TXT

convert_to_txt("assets/colmap/workspaces/eastbound/sparse_aligned", "assets/colmap/reconstruction/eastbound/sparse_aligned")

In [ ]:
# Eastbound 
colmap_reconstruction(COLMAP_WORKSPACE_EASTBOUND, UNDISTORTED_EASTBOUND_PATH )
convert_to_txt(COLMAP_WORKSPACE_EASTBOUND, COLMAP_TXT_RECON_EASTBOUND )

# Westbound
colmap_reconstruction(COLMAP_WORKSPACE_WESTBOUND, UNDISTORTED_WESTBOUND_PATH )
convert_to_txt(COLMAP_WORKSPACE_WESTBOUND, COLMAP_TXT_RECON_WESTBOUND )

In [ ]:
!mkdir -p ./assets/workspaces/eastbound
!colmap automatic_reconstructor --workspace_path ./assets/workspaces/eastbound --image_path ./assets/undistorted_05/eastbound --sparse 1 --dense 0 --single_camera 1

### Align model orientation

In [ ]:
!mkdir assets\colmap\workspaces\eastbound\sparse_aligned

!colmap model_orientation_aligner \
        --input_path assets/colmap/workspaces/eastbound/sparse/0 \
        --output_path assets/colmap/workspaces/eastbound/sparse_aligned \
        --image_path assets/undistorted_05_short \
        --method MANHATTAN-WORLD

!colmap model_orientation_aligner --input_path assets/colmap/workspaces/eastbound/sparse/0 --output_path assets/colmap/workspaces/eastbound/sparse_aligned --image_path assets/undistorted_05_short --method MANHATTAN-WORLD

# I tried the IMAGE-ORIENTATION method aswel but it doesn't work very well
# Colmap has a weird quirk where the Y-axis points downwards. This is by design appearantly.
# https://medium.com/red-buffer/mastering-3d-spaces-a-comprehensive-guide-to-coordinate-system-conversions-in-opencv-colmap-ef7a1b32f2df

In [ ]:
# Convert results to txt files (optional)
!mkdir ../assets/workspace/sparse_aligned_man_txt
!colmap model_converter --input_path $workspace/sparse_aligned_man --output_path $workspace/sparse_aligned_man_txt --output_type TXT

In [ ]:
convert_to_txt('./assets/temp/sparse_aligned', './assets/temp/sparse_aligned_txt' )

In [ ]:
!mkdir assets/colmap/workspaces/eastbound/dense_aligned

#assets\colmap\workspaces\eastbound\dense

"""!colmap model_orientation_aligner \
        --input_path assets/colmap/workspaces/eastbound/dense \
        --output_path assets/colmap/workspaces/eastbound/dense_aligned \
        --image_path assets/undistorted_05 \
        --method MANHATTAN-WORLD"""

# Tree Mapping

In [ ]:

def load_images(file_path):
    images = {}
    with open(file_path, 'r') as f:
        while True:
            line = f.readline()
            if not line:
                break
            if line.startswith('#'):
                continue
            elements = line.split()
            image_id = int(elements[0])
            qw, qx, qy, qz = map(float, elements[1:5])
            tx, ty, tz = map(float, elements[5:8])
            camera_id = int(elements[8])
            image_name = elements[9]

            line = f.readline()
            points2d = []
            elements = line.split()
            for i in range(0, len(elements), 3):
                x, y, point3d_id = map(float, elements[i:i+3])
                points2d.append([x, y, int(point3d_id)])
            points2d = np.array(points2d)

            images[image_name] = {
                'image_id': image_id,
                'qw': qw, 'qx': qx, 'qy': qy, 'qz': qz,
                'tx': tx, 'ty': ty, 'tz': tz,
                'camera_id': camera_id, 'image_name': image_name, 'points2d': points2d
            }
    return images

def load_points3d(file_path):
    points3d = {}
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            elements = line.split()
            point_id = int(elements[0])
            xyz = np.array(elements[1:4], dtype=float)
            rgb = np.array(elements[4:7], dtype=int)
            error = float(elements[7])
            track = np.array(elements[8:], dtype=int).reshape(-1, 2)
            points3d[point_id] = {'xyz': xyz, 'rgb': rgb, 'error': error, 'track': track}
    return points3d

def find_corresponding_3d_points(image_data, keypoints_2d, points3d):
    corresponding_3d_points = []
    for kp in keypoints_2d:
        x, y = kp
        distances = np.linalg.norm(image_data['points2d'][:, :2] - np.array([x, y]), axis=1)
        closest_point_idx = np.argmin(distances)
        point2d_id = image_data['points2d'][closest_point_idx, 2]
        if point2d_id != -1:
            corresponding_3d_points.append(points3d[point2d_id]['xyz'])
        else:
            corresponding_3d_points.append(None)
    return corresponding_3d_points

def interpolate_missing_point(p1, p2):
    if p1 is None and p2 is not None:
        return p2
    if p1 is not None and p2 is None:
        return p1
    if p1 is not None and p2 is not None:
        return (np.array(p1) + np.array(p2)) / 2
    return None

def ensure_all_points_mapped(corresponding_3d_points):
    n = len(corresponding_3d_points)
    for i in range(n):
        if corresponding_3d_points[i] is None:
            left = None
            right = None
            if i > 0:
                left = corresponding_3d_points[i-1]
            if i < n-1:
                right = corresponding_3d_points[i+1]
            corresponding_3d_points[i] = interpolate_missing_point(left, right)
    return corresponding_3d_points

def calculate_distance_3d(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))


In [ ]:
def k_closest_points(points, target_point, k=7):
    """
    Finds the k closest points to a target point.

    Args:
    points (list of tuples): List of 2d point (x, y, _) points.
    target_point (tuple): The (x, y) coordinates of the target point.
    k (int, optional): Number of closest points to find. Default is 7.

    Returns:
    list of tuples: The k closest points including their additional data.
    """
    temp = [(x, y) for x, y, _ in points]

    distances = np.linalg.norm(temp - target_point, axis=1)
    
    k_indices = np.argsort(distances)[:k]
    
    return points[k_indices]


def get_closest_3d_points_for_felling_cut(points2d, points3d, felling_cut):
    """
    Finds the closest 3D points to a felling cut based on 2D points.

    Args:
    points2d (list of tuples): List of (x, y, id) points.
    points3d (list of tuples): List of 3D points corresponding to the ids in points2d.
    felling_cut (tuple): The (x, y) coordinates of the felling cut.

    Returns:
    list of tuples: The 3D points closest to the felling cut.
    """
    closest_points = k_closest_points(np.array(points2d), np.array(felling_cut))

    closest_points_3d = []

    for point in closest_points:
        punt3d_id = point[2]

        closest_points_3d.append(points3d[int(punt3d_id)])

    return closest_points_3d


def get_closest_3d_points_for_image(points2d, points3d, pred_keypoints):
    """
    Finds the closest 3D points for each felling cut in image.

    Args:
    points2d (list of tuples): List of (x, y, id) points.
    points3d (list of tuples): List of 3D points corresponding to the ids in points2d.
    pred_keypoints (list of lists): List of predicted keypoints, each containing (x, y) coordinates.

    Returns:
    list of lists: Each inner list contains the 3D points closest to the corresponding felling cut.
    """
    closest_points = []
    
    for treepoints in pred_keypoints:
        felling_cut = treepoints[0]

        felling_point = (felling_cut[0], felling_cut[1])

        closest_points_3d = get_closest_3d_points_for_felling_cut(points2d, points3d, felling_point)

        closest_points.append(closest_points_3d)

    return closest_points


def generate_random_rgb_values(num_values = 250):
    """
    Generates an array of random RGB values.

    Args:
    num_values (int): The number of RGB values to generate.

    Returns:
    numpy.ndarray: An array of shape (num_values, 3) with random RGB values between 0 and 1.
    """
    rgb_values = np.random.rand(num_values, 3)
    return rgb_values

def is_eastbound(name):
    """
    This function checks if the given string contains the substring "east".
    
    Parameters:
    name (str): The string to be checked for the presence of "east".
    """
    if "eastbound" in name:
        return True
    else:
        return False
    
def get_json_data(json_path):
    """
    This function reads a JSON file from the specified path and returns the data as a dictionary.
    
    Parameters:
    json_path (str): The path to the JSON file to be read.
    
    Returns:
    dict: The data contained in the JSON file.
    """
    with open(json_path, 'r') as file:
        data = json.load(file)

    return data

def get_filtered_points(points2d):
    """
    This function filters out 2D points that do not have a corresponding 3D point.
    
    It assumes that the third element in each point (i.e., point[2]) indicates whether 
    there is a corresponding 3D point. If point[2] is greater than -1, the point has 
    a corresponding 3D point and is included in the filtered list.

    Parameters:
    points2d (list of lists): A list of 2D points where each point is represented as 
                              a list with at least three elements.

    Returns:
    list of lists: A list of 2D points that have corresponding 3D points.
    """
    filtered_points2d = []
    for point in points2d:
        if int(point[2]) > -1:
            filtered_points2d.append(point)

    return filtered_points2d

def getNumber(x):
    num_w_ext = x.split('_')[2]
    num = num_w_ext.split('.')[0]
    return num


def take_median_of_closest_3d_points(closest_points, median_list):
    """
    This function calculates the median of the closest 3D points for each set of points in the input list and 
    appends the median to a provided list.

    Parameters:
    closest_points (list of lists of dicts): A list where each element is a list of dictionaries. Each dictionary 
                                             represents a point and contains a key 'xyz' with the 3D coordinates.
    median_list (list): A list to which the calculated median of the 3D points will be appended.
    """
    for tree in closest_points:
        # calculate
        points_to_calculate = []
        for point in tree:
            points_to_calculate.append(point['xyz'])

        mean_array = np.median(points_to_calculate, axis=0)
        median_list.append(mean_array)

def process_images_and_compute_felling_cut_median(files, images, points3d):
    """
    This function processes a list of image files, computes the median of the closest 3D points to the felling cuts 
    for each image, and returns a list of these medians.

    Parameters:
    files (list of str): A list of image file names to be processed.
    images (dict): A dictionary where keys are image paths and values are dictionaries containing 'points2d' 
                   corresponding to each image.

    Returns:
    list: A list containing the median of the closest 3D points for the felling cuts of each image.

    Detailed Steps:
    1. Initialize an empty list `median_all_felling_cuts_list` to store the medians.
    2. Iterate over the sorted list of image files using `tqdm` for progress display.
    3. For each image, replace the file extension '.png' with '.json' to get the corresponding JSON file name.
    4. Determine if the image is eastbound or westbound using the `is_eastbound` function.
    5. Load the JSON data containing predicted keypoints from the appropriate directory.
    6. Retrieve the 2D points for the current image from the `images` dictionary.
    7. Filter out 2D points that do not have corresponding 3D points using the `get_filtered_points` function.
    8. Compute the closest 3D points to all felling cuts of the current image using the `get_closest_3d_points_for_image` function.
    9. Calculate the median of these closest 3D points using the `take_median_of_closest_3d_points` function and append it to `median_all_felling_cuts_list`.
    10. Return the list of medians.

    Example:
    >>> files = ["image1.png", "image2.png"]
    >>> images = {
    ...     "eastbound/image1.png": {'points2d': [[1, 2, 3], [4, 5, 6]]},
    ...     "westbound/image2.png": {'points2d': [[7, 8, 9], [10, 11, 12]]}
    ... }
    >>> median_all_felling_cuts_list = process_images_and_compute_felling_cut_median(files, images)
    >>> print(median_all_felling_cuts_list)
    [array([...]), array([...])]
    """
    median_all_felling_cuts_list = []

    for i, image in enumerate(tqdm(sorted(files), desc="Processing images")):
        
        # get pred keypoints
        json_name = image.replace('.png', '.json')

        if (is_eastbound(image)):
            data = get_json_data(f"{ANNOTATED_EASTBOUND}{json_name}")

            # get points2d for current image
            # TODO: Something weird going on here.
            #points2d = images[image]['points2d']
            points2d = images[f"eastbound/{image}"]['points2d']
        else:
            data = get_json_data(f"{ANNOTATED_WESTBOUND}{json_name}")

            # get points2d for current image
            points2d = images[image]['points2d']
        
        filtered_points2d = get_filtered_points(points2d)

        # get closest points to all felling cuts of this image
        closest_3d_points = get_closest_3d_points_for_image(filtered_points2d, points3d, data["pred_keypoints"])

        # take median of all these points
        take_median_of_closest_3d_points(closest_3d_points, median_all_felling_cuts_list)

    return median_all_felling_cuts_list



def convert_points_to_df(points):
    X = [item[0] for item in points]
    Y = [item[1] for item in points]
    Z = [item[2] for item in points]

    df = pd.DataFrame({'X': X, 'Y': Y, 'Z': Z})
    return df


def cluster_points(points, eps=0.1, min_samples=3, with_outliers=False):
    """
    This function clusters 3D points using the DBSCAN (Density-Based Spatial Clustering of Applications with Noise) 
    algorithm and returns the clustered points in a pandas DataFrame.

    Parameters:
    points (list of lists or tuples): A list where each element is a list or tuple representing a 3D point with 
                                      three coordinates (X, Y, Z).
    eps (float): The maximum distance between two samples for one to be considered as in the neighborhood of the other.
                 This is not a maximum bound on the distances of points within a cluster. Default is 0.1.
    min_samples (int): The number of samples (or total weight) in a neighborhood for a point to be considered 
                       as a core point. This includes the point itself. Default is 3.
    with_outliers (bool): If True, the function returns all points including outliers. If False, outliers are removed.
                          Default is False.

    Returns:
    pandas.DataFrame: A DataFrame containing the coordinates of the input points and their corresponding cluster labels.
                      If `with_outliers` is False, the DataFrame will not include points classified as outliers.

    Example:
    >>> points = [(1, 2, 3), (4, 5, 6), (7, 8, 9), (10, 11, 12)]
    >>> clustered_df = cluster_points(points, eps=0.5, min_samples=2, with_outliers=True)
    >>> print(clustered_df)
        X   Y   Z  Cluster
    0   1   2   3        0
    1   4   5   6        0
    2   7   8   9        1
    3  10  11  12       -1

    Note:
    - The `Cluster` column contains the cluster labels assigned by DBSCAN. 
    - Points labeled as -1 are considered outliers by the DBSCAN algorithm.
    """
    dataframe = convert_points_to_df(points)
    
    X = dataframe[['X', 'Y', 'Z']]

    print(X.head())

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_scaled = X

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    clusters = dbscan.fit_predict(X_scaled)

    dataframe['Cluster'] = clusters

    if(with_outliers):
        return dataframe

    # df_no_outlier = dataframe[dataframe['Cluster'] != -1]
    return dataframe

In [ ]:
#images = load_images('./assets/temp/sparse_aligned_txt/images.txt')
#points3d = load_points3d('./assets/temp/sparse_aligned_txt/points3D.txt')

images = load_images(f"{COLMAP_TXT_RECON_EASTBOUND}/sparse_aligned/images.txt")
points3d = load_points3d(f"{COLMAP_TXT_RECON_EASTBOUND}/sparse_aligned/points3D.txt")


In [ ]:
# This cell goes through all images
#     takes for each image all trees, 
#         calculates for each tree the closest points to that felling cut
#         map these closest points to the 3d space
#         take the median of these 3 points
# Cluster all the median 3d points


#IMG_LOC  = "./assets/temp/eastbound/*.png"
IMG_LOC  = "assets/annotated_05/eastbound/*.png"
COLORS   = generate_random_rgb_values()

file_paths = glob(IMG_LOC)

files = [os.path.basename(file_path) for file_path in file_paths]

sorted_files = sorted(files, key=lambda x: int(getNumber(x)))

median_all_felling_cuts_list = process_images_and_compute_felling_cut_median(sorted_files[:150], images, points3d)

df_clustered_points = cluster_points(median_all_felling_cuts_list)


# Tree Measurement

### Functions

In [ ]:
def get_id_and_3D_point_list(image):
    """
    Calculates 3D median point + clusterid of the felling cut of every tree on the image

    Parameters:
    image (str) : filename of an image that has been annotated (only filename, no path)

    Returns:
    List: List of DBScan id's (int) in the order of the trees in the json file
    List: List of 3D median points in the order of the trees in the json file

    Note:
    DBScan id is -4 if no matching 3D median point is found for the felling cut
    DBScan id is -1 if the 3D median point is not part of a cluster
    """
    
    number = getNumber(image)

    json_name = image.replace('.png', '.json')

    data = get_json_data(ANNOTATED_EASTBOUND + json_name)

    pred_keypoints = data["pred_keypoints"]

    # get points2d
    if f"eastbound/{image}" not in images.keys():
        print("image not used in sparse map")
        return (None, None)
            

    points2d_img = images[f"eastbound/{image}"]     ## images?
    #points2d_img = images[f"{image}"]     ## images?

    points2d = points2d_img['points2d']

    filtered_points2d = get_filtered_points(points2d)

    # get closest points
    closest_3d_points = get_closest_3d_points_for_image(filtered_points2d, points3d, pred_keypoints)

    # TAKE MEDIAN/MEAN
    id_list = []
    point_3d_list = []

    for index, tree in enumerate(closest_3d_points):
        points_to_calculate = []

        for point in tree:
            points_to_calculate.append(point['xyz'])

        mean_array = np.median(points_to_calculate, axis=0)
        point_3d_list.append(mean_array)

        filtered_df = df_clustered_points[(df_clustered_points['X'] == mean_array[0]) & (df_clustered_points['Y'] == mean_array[1]) & (df_clustered_points['Z'] == mean_array[2])]
        """round_nr = 0
        x_eq = round(df_clustered_points['X'], round_nr) == round(mean_array[0], round_nr)
        y_eq = round(df_clustered_points['Y'], round_nr) == round(mean_array[1], round_nr)
        z_eq = round(df_clustered_points['Z'], round_nr) == round(mean_array[2], round_nr)
        filtered_df = df_clustered_points[x_eq & y_eq & z_eq]"""

        if filtered_df.shape[0] != 0:
            cluster = filtered_df.iloc[0].Cluster
            id_list.append(cluster)
        else:
            cluster = -4
            id_list.append(cluster)

    return id_list, point_3d_list

In [ ]:
def get_depth_normal_map_paths(file_name, workspace):
    depth_map_path = f"{workspace}dense/stereo/depth_maps/eastbound/{file_name}"  # Change this to your actual file path
    normal_map_path = f"{workspace}dense/stereo/normal_maps/eastbound/{file_name}"  # Change this to your actual file path
    return depth_map_path, normal_map_path

def get_depth_map(frame_number, workspace, orig_width=2704 , orig_height=1520, verbose=False):
    """
    Loads and preprocesses a depth image for a frame number.

    Args:
    frame_number (int) : current frame number
    workspace (string) : path of current workspace
    orig_width (int, opt) : width of undistorted frames
    orig_height (int, opt) : height of undistorted frames

    Details:
    1. Create file paths for depth and normal maps, check if exists
    2. Read depth map from png.geometric.bin file -> np.ndarray
    3. Clip depth values with 5 and 95 percentile depth value
    4. Resize depth image so depth image size is equal to the dimensions of the real image
    5. Replace every pixel of image with (depth > 5 units OR depth = 0) with np.NaN value for better visualization

    Returns:
    ndarray with dims (height*, width*) filled with dtype np.float32.   (* from original image, not depth image)
    """
    # Set the file paths for the depth and normal maps

    # TODO: change this to a more standardized file_name?
    file_name = f"eastbound_20240319_{frame_number}.png.geometric.bin"

    depth_map_path, normal_map_path = get_depth_normal_map_paths(file_name, workspace)

    # Check if files exist
    if not os.path.exists(depth_map_path):
        raise FileNotFoundError(f"File not found: {depth_map_path}")

    if not os.path.exists(normal_map_path):
        raise FileNotFoundError(f"File not found: {normal_map_path}")
    
    def read_array(path):
        """
        Util function to extract depth or normal image from png.geometric.bin file
        """    
        with open(path, "rb") as fid:
            width, height, channels = np.genfromtxt(
                fid, delimiter="&", max_rows=1, usecols=(0, 1, 2), dtype=int
            )
            fid.seek(0)
            num_delimiter = 0
            byte = fid.read(1)
            while True:
                if byte == b"&":
                    num_delimiter += 1
                    if num_delimiter >= 3:
                        break
                byte = fid.read(1)
            array = np.fromfile(fid, np.float32)
        array = array.reshape((width, height, channels), order="F")
        return np.transpose(array, (1, 0, 2)).squeeze()

    # Read depth and normal maps
    depth_map = read_array(depth_map_path)
    # normal_map = read_array(normal_map_path)

    # Set the visualization parameters
    min_depth_percentile = 5
    max_depth_percentile = 95

    # Process the depth map based on percentiles
    min_depth, max_depth = np.percentile(depth_map, [min_depth_percentile, max_depth_percentile])
    if verbose:
        print(f"min_depth: {min_depth}, max_depth: {max_depth}")
    depth_map[depth_map < min_depth] = min_depth
    depth_map[depth_map > max_depth] = max_depth

    # Resize the depth map to match the dimensions of the original image
    # depth_map_resized = cv2.resize(depth_map, (original_image.shape[1], original_image.shape[0]))
    depth_map_resized = cv2.resize(depth_map, (orig_width, orig_height), interpolation=cv2.INTER_NEAREST)

    depth_map = depth_map_resized

    # Remove values that are too far from the camera and also set 0 values to NaN
    # Replace zeros with NaNs for better visualization
    depth_map[depth_map == 0] = np.nan
    depth_map[depth_map > 5] = np.nan
    
    return depth_map

In [ ]:
def compute_3d_coordinates_simple_radial(x, y, depth, fx, cx, cy, k1):
    """
    Convert pixel coordinates to world coordinates using the depth value and camera intrinsics.
    """

    # Assuming distortion is negligible for simplicity.
    X = (x - cx) * depth / fx
    Y = (y - cy) * depth / fx
    Z = depth

    X = (x - cx) * depth / fx
    Y = (y - cy) * depth / fx
    Z = depth / (1 + k1 * depth)
    # return X, Y, Z
    return np.array([X, Y, Z])

def compute_tree_trunk_width(left_point, right_point, depth_map, fx, cx, cy, k1, depthm_override=False, d1=None, d2=None, verbose=False):
    """
    Returns width of between two 2D points on the frame
    """
    
    x1, y1 = left_point
    x2, y2 = right_point
    
    mid_point = ((x1 + x2) // 2, (y1 + y2) // 2)
    xm = mid_point[0]
    ym = mid_point[1]
    
    depth1 = depth_map[y1, x1]
    depth2 = depth_map[y2, x2]
    depthm = depth_map[ym, xm]
    if verbose:
        print(depth1, depthm, depth2)
    if depthm_override:
        if d1 is not None:
            depth1 = d1
        else:
            depth1 = depthm
        if d2 is not None:
            depth2 = d2
        else:
            depth2 = depthm   
        if verbose:     
            print(depth1, depthm, depth2)
    # depthm = depth_map[700,1650]
    # print(depth1, depth2, depthm)
    
    point1_3d = compute_3d_coordinates_simple_radial(x1, y1, depth1, fx, cx, cy, k1)
    point2_3d = compute_3d_coordinates_simple_radial(x2, y2, depth2, fx, cx, cy, k1)
    
    if verbose:
        print(point1_3d, point2_3d)
        print(point1_3d - point2_3d)
    
    width = np.linalg.norm(point1_3d - point2_3d)
    return width

In [ ]:
def calculate_width_between_keypoints(current_tree_keypoints, depth_map, units_per_meter, fx, cx, cy, k1, visualize_data=False, verbose=False):
    """
    Calculates width of single perceptree tree on a frame in units and meters
    Returns a dictionary
    """
    
    middle_y_value = (current_tree_keypoints[1][1] + current_tree_keypoints[2][1]) / 2
    left_point = current_tree_keypoints[1][0], current_tree_keypoints[1][1]
    right_point = current_tree_keypoints[2][0], current_tree_keypoints[2][1]   
        
    left_point = (int(current_tree_keypoints[1][0]), int(middle_y_value))
    right_point = (int(current_tree_keypoints[2][0]), int(middle_y_value))
    if verbose:
        print(left_point, right_point)
    
    ### For visualisations, see tree_width_estimation notebook
    # if visualize_data:        
    #     depth_values, xm,y1 = extract_depth_along_Y_axis(depth_map, left_point, right_point)
    #     plot_points_Y_axis(depth_values, xm,y1)
        
    #     depth_values = extract_depth_along_line(depth_map, left_point, right_point, 20)
    #     plot_points(depth_values, middle_y_value, left_point[0])
    
    
    width_units = compute_tree_trunk_width(left_point, right_point, depth_map, fx, cx, cy, k1)
    width_cm = (width_units / units_per_meter) * 100
    if verbose:
        print(f"The width of the tree trunk is: {width_units} units")
        print(f"The width of the tree trunk is: {width_cm} centimeters")

    width_units_mid = compute_tree_trunk_width(left_point, right_point, depth_map, fx, cx, cy, k1, depthm_override=True)
    width_cm_mid = (width_units_mid / units_per_meter) * 100
    if verbose:
        print(f"The width of the tree trunk is: {width_units_mid} units - using mid depth")
        print(f"The width of the tree trunk is: {width_cm_mid} centimeters - using mid depth") 
    
    row_data = {
        'width_units': width_units,
        'width_cm': width_cm,
        'width_units_mid': width_units_mid,
        'width_cm_mid': width_cm_mid
    }
    return row_data

### Setup

In [ ]:
#
# Creating a list of frame numbers that have been used when creating the dense/sparse model
#

# TODO: clean this up, this can be prettier code

depth_dir_pattern = f"{COLMAP_WORKSPACE_EASTBOUND}/dense/stereo/depth_maps/*/*.png.geometric.bin"

depth_paths = glob(depth_dir_pattern)
depth_paths.sort()

frame_numbers = []

for i in range(len(depth_paths)):
    dotsplit = depth_paths[i].split('.')
    frame_number = dotsplit[-4].split('_')[-1]
    frame_numbers.append(frame_number)

print(frame_numbers)

In [ ]:
# TODO: Camera intrinsices hardcoded, link the camera intrinsics with the ones obtained before.

# Camera intrinsics
fx = 1463.2
cx = 1352
cy = 760
k1 = 0.01502

# TODO: create option here to do calibration manually? Display current calibration.

unit_calc_frame_nr = "07851"
workspace = COLMAP_WORKSPACE_EASTBOUND
unit_calc_depth_map = get_depth_map(unit_calc_frame_nr, workspace)

right_point_road = (1650,1000)
left_point_road = (460,950)

road_width = compute_tree_trunk_width(left_point_road, right_point_road, unit_calc_depth_map, fx, cx, cy, k1, depthm_override=False)
print(f"The width of the road is: {road_width} units")
print(f"The width of the road is: 4m")
units_per_meter = road_width / 4

print(f"\t-> units per meter: {units_per_meter}")

### Big for-loop

In [ ]:
"""
Loops through every frame with a depth map, calculates the widths of the trees in the frame, 
    matches trees on frame with trees in dbscan results and calculates median + std depth accross all frames.
Result is saved in csv.
"""

import json

width_measurements_file = 'tree_widths_3.csv'

#if not os.path.isfile(width_measurements_file):
if True:
    # Note: Visualizing data is not implemented in the notebook, See tree_width_estimation for some visual examples of the workflow
    visualize_data = False
    verbose = False

    tree_list = []

    tree_id = 1

    for i in tqdm(range(len(frame_numbers))):
        depth_map = get_depth_map(frame_numbers[i], COLMAP_WORKSPACE_EASTBOUND)
        if verbose:
            print("Currently reading file:")
        json_name = f"eastbound_20240319_{frame_numbers[i]}.json"
        js = f"{ANNOTATED_EASTBOUND}{json_name}"
        # js = json_paths[i]

        with open(js, 'r') as js:
            data = json.load(js)
            
        # Sort the data by the x-coordinate of the left keypoint
        sorted_indices = sorted(range(len(data['pred_boxes'])), key=lambda i: data['pred_keypoints'][i][1][0])
        sorted_pred_keypoints = [data['pred_keypoints'][i] for i in sorted_indices]
        if verbose:
            print(sorted_pred_keypoints)
        
        #if visualize_data:     
        #    visualize_depth_map(depth_map, frame_numbers[i], sorted_pred_keypoints)

        #
        #
        #
        id_list, point_3D_list = get_id_and_3D_point_list(json_name.replace('.json', '.png'))
        sorted_id_list = [id_list[i] for i in sorted_indices]
        sorted_point_3D_list = [point_3D_list[i] for i in sorted_indices]
        #
        #
        # 
         
        number_of_trees = len(sorted_pred_keypoints)
        
        for j in range(number_of_trees):        
            tree_trunk_number = j + 1

            width_row_data = calculate_width_between_keypoints(sorted_pred_keypoints[j], depth_map, units_per_meter, fx, cx, cy, k1, visualize_data=visualize_data, verbose=verbose)
            
            # Create a dictionary with the data
            row_data = {
                'tree_id': sorted_id_list[j],
                'X' : sorted_point_3D_list[j][0],
                'Y' : sorted_point_3D_list[j][1],
                'Z' : sorted_point_3D_list[j][2],
                #'json_name': json_name, # this is not necessary in end result
                'frame_number': frame_numbers[i],
                'number_of_trees': number_of_trees,
                'tree_trunk_number': tree_trunk_number,
            }
            row_data.update(width_row_data)
            
            # Append the dictionary to the list
            tree_list.append(row_data)
            
            tree_id+=1
            
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(tree_list)

    df.to_csv(width_measurements_file, index=True)

In [ ]:
df = pd.read_csv(width_measurements_file)

cols = ['width_cm_mid'] # one or more

Q1 = df[cols].quantile(0.25)
print(type(Q1))
print(Q1.shape)
Q3 = df[cols].quantile(0.75)
IQR = Q3 - Q1

inliers_df = df[~((df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]
outliers_df = df[~((df[cols] > (Q1 - 1.5 * IQR)) |(df[cols] < (Q3 + 1.5 * IQR))).any(axis=1)]

#df = inliers_df
print(df.shape)


#df.head(20)

#df.groupby(['tree_id'], as_index=False).agg({'width_units':['mean','std'],'width_cm':['mean','std'],'width_units_mid':['mean','std'],'width_cm_mid':['mean','std']})
g_df = df.groupby(['tree_id'], as_index=False).agg({'width_cm':['mean','std'],'width_cm_mid':['mean','std'], 'number_of_trees':['count']})

#print(outliers_df.shape)
#outliers_df

#df[(df.width_cm_mid > 80) | (df.width_cm_mid < 5)]

In [ ]:
strict_df = df[(df.width_cm_mid < 100) & (df.width_cm_mid > 5)]

strict_df = strict_df.groupby(['tree_id'], as_index=False).agg({'width_cm':['mean','std'],'width_cm_mid':['mean','std'], 'number_of_trees':['count']})

In [ ]:
compare = g_df.join(strict_df, on="tree_id", lsuffix='_before', rsuffix='_after')[["tree_id", "width_cm_mid_before", "width_cm_mid_after"]]

compare["diff"] = compare["width_cm_mid_after"]["std"] - compare["width_cm_mid_before"]["std"]
print(sum(compare["diff"].dropna()))
compare

# Output

In [ ]:
df_clustered_points_no_outliers = df_clustered_points[df_clustered_points['Cluster'] != -1]

df_map_to_process = df_clustered_points_no_outliers.copy()

df_map_to_process['Y'] = 0

df_map = df_map_to_process.groupby('Cluster').median().reset_index()

df_map

# Visualisation

In [ ]:

#files = os.listdir(ANNOTATED_EASTBOUND)

files = [os.path.basename(x) for x in glob(f"{ANNOTATED_EASTBOUND}*.png")]

sorted_files = sorted(files, key=lambda x: int(getNumber(x)))

# Generate 250 random RGB values
colors = generate_random_rgb_values(250)


another_list = []

for index, image in tqdm(enumerate(sorted_files[:20])):

    number = getNumber(image)

    mask_path = image.replace('.png', '_pred_mask.npy')

    masks = np.load(f"./assets/annotated_05/eastbound/{mask_path}", allow_pickle=True)

    json_name = image.replace('.png', '.json')

    data = get_json_data(f"./assets/annotated_05/eastbound/{json_name}")
    pred_keypoints = data["pred_keypoints"]

    # get points2d
    points2d_img = images[f"eastbound/{image}"]
    points2d = points2d_img['points2d']
    filtered_points2d = get_filtered_points(points2d)

    # get closest points
    closest_3d_points = get_closest_3d_points_for_image(filtered_points2d, points3d, pred_keypoints)
    

    image_path = f"./assets/undistorted_05/eastbound/{image}"  # Update with the path to your image file

    img = cv2.imread(image_path)

    # TAKE MEDIAN/MEAN
    id_list = []

    for index, tree in enumerate(closest_3d_points):
        points_to_calculate = []

        for point in tree:
            points_to_calculate.append(point['xyz'])

        mean_array = np.median(points_to_calculate, axis=0)

        filtered_df = df_clustered_points[(df_clustered_points['X'] == mean_array[0]) & (df_clustered_points['Y'] == mean_array[1]) & (df_clustered_points['Z'] == mean_array[2])]

        id_list.append(filtered_df.iloc[0].Cluster)
        

    for index, pred_box in enumerate(data["pred_boxes"]):
        if(id_list[index] != -1):
            start_point = (int(pred_box[0]), int(pred_box[1]))
            end_point = (int(pred_box[2]), int(pred_box[3]))
            color = [int(c * 255) for c in COLORS[int(id_list[index]) % len(COLORS)]]
            thickness = 5

            cv2.rectangle(img, start_point, end_point, (color[2], color[1], color[0]) , thickness)

            mask = masks[index] * 255
            color_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
            color_mask[:,:,0] = (mask * color[2] * 255).astype(np.uint8)
            color_mask[:,:,1] = (mask * color[1] * 255).astype(np.uint8) 
            color_mask[:,:,2] = (mask * color[0] * 255).astype(np.uint8)


            # Apply the mask onto the image
            img = cv2.addWeighted(img, 1, color_mask, 0.5, 0)

            font = cv2.FONT_HERSHEY_SIMPLEX
            text = str(int(id_list[index]))
            text_size = cv2.getTextSize(text, font, 1, 2)[0]
            text_x = int(pred_box[0] + (pred_box[2] - pred_box[0]) // 2 - text_size[0] // 2)
            text_y = int( pred_box[1] + (pred_box[3] - pred_box[1]) // 2 + text_size[1] // 2)
            
            # Add a white background for the text
            bg_top_left = (text_x - 5, text_y + 5)
            bg_bottom_right = (text_x + text_size[0] + 5, text_y - text_size[1] - 5)
            cv2.rectangle(img, bg_top_left, bg_bottom_right, (255, 255, 255), cv2.FILLED)

            cv2.putText(img, text, (text_x, text_y), font, 1, (0, 0, 0), 2, cv2.LINE_AA)


    cv2.imwrite(f"temp_output/output_{number}.png", img)


    plt.figure(figsize=(6, 6))

    # Loop through each row in the median DataFrame and plot individually
    for index, row in df_map.iterrows():
        cluster = row['Cluster']
        color = COLORS[int(cluster % len(COLORS))]
        plt.scatter(row['Z'], -row['X'], color=color, s=100, label=f'Cluster {cluster}')


    cam_x = points2d_img['tx']
    cam_z = - points2d_img['tz']

    plt.scatter(cam_x, cam_z, s=50, color="red")
    plt.xlim(cam_x -2, cam_x + 2)
    plt.ylim(cam_z -2, cam_z + 2)

    plt.axis('off')

    plt.savefig(f"temp_plots/plot_image_{number}.png")


    plot = cv2.imread(f"temp_plots/plot_image_{number}.png", cv2.IMREAD_UNCHANGED)
    tree_img = cv2.imread(f"temp_output/output_{number}.png", cv2.IMREAD_UNCHANGED)


    plot = plot[:, :, :3]

    # Resize images to have the same height (assuming you want them to have the same height)
    height = max(plot.shape[0], tree_img.shape[0])
    plot_resized = cv2.resize(plot, (int(plot.shape[1] * height / plot.shape[0]), height))
    tree_img_resized = cv2.resize(tree_img, (int(tree_img.shape[1] * height / tree_img.shape[0]), height))

    # Concatenate images horizontally
    final_image = np.concatenate((tree_img_resized, plot_resized), axis=1)

    # Save or display the final image
    cv2.imwrite(f"testing/output_{number}.png", final_image)



In [ ]:
def create_video_from_images(image_folder, output_video, fps=30):
    # Get all image file paths from the folder
    image_paths = sorted(glob(os.path.join(image_folder)))  # Change '*.jpg' if your images have a different extension
    if not image_paths:
        print("No images found in the folder.")
        return

    # Read the first image to get the frame size
    frame = cv2.imread(image_paths[0])
    height, width, _ = frame.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for mp4
    video_writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    for image_path in image_paths:
        frame = cv2.imread(image_path)
        video_writer.write(frame)  # Write the frame to the video

    # Release the video writer object
    video_writer.release()
    print(f"Video saved to {output_video}")


image_folder = './testing/*.png'
output_video = 'output_video_eastbound_short.mp4'
create_video_from_images(image_folder, output_video, fps=8)
